###Prepare environment

In [1]:
!pip install transformers --quiet
!pip install datasets[audio] --quiet
!pip install evaluate --quiet
!pip install jiwer --quiet
!pip install gradio --quiet
!pip install transformers[torch] --quiet

In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline, WhisperTokenizer, WhisperFeatureExtractor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, load_dataset, Audio, DatasetDict
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from huggingface_hub import notebook_login
import evaluate

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug 22 19:54:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [5]:
pretrained = "openai/whisper-tiny"

In [48]:
notebook_login()

###Load dataset

In [11]:
dataset = DatasetDict()
dataset["train"] = load_dataset("jpdiazpardo/guturalScream_metalVocals", split='train')
dataset["test"] = load_dataset("jpdiazpardo/guturalScream_metalVocals", split='test')


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1650 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/551 [00:00<?, ? examples/s]

In [12]:
print(dataset)

In [13]:
dataset = dataset.remove_columns(["song_name", 'artist_name', 'album_name', 'release_year', 'video_id', 'timestamp_start', 'timestamp_end', 'sample_rate'])

###Prepare feature extractor, tokenizer and Data

####Load WhisperFeatureExtractor

In [14]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(pretrained)

####Load WhisperTokenizer

In [15]:
tokenizer = WhisperTokenizer.from_pretrained(pretrained, language="English", task="transcribe")

Combine To Create A WhisperProcessor

In [16]:
processor = WhisperProcessor.from_pretrained(pretrained, language="English", task="transcribe")

#### Prepare data

In [17]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [18]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [19]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/1650 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/551 [00:00<?, ? examples/s]

###Training and Evaluation

####Define a data collector

In [20]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

####Evaluation Metrics

In [22]:
metric = evaluate.load("wer")

In [23]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

####Load a Pre-Trained Checkpoint

In [24]:
model = WhisperForConditionalGeneration.from_pretrained(pretrained)

In [25]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

####Define the Training Configuration

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir="jpdiazpardo/whisper-tiny-metal",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,

)

In [32]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [33]:
processor.save_pretrained(training_args.output_dir)

In [34]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
1000,0.084400,1.396659,104.605456
2000,0.001800,1.586115,93.546495
3000,0.000800,1.655185,78.938105
4000,0.000600,1.677979,79.906131


TrainOutput(global_step=4000, training_loss=0.30561430572066456, metrics={'train_runtime': 9584.2288, 'train_samples_per_second': 6.678, 'train_steps_per_second': 0.417, 'total_flos': 1.56251140079616e+18, 'train_loss': 0.30561430572066456, 'epoch': 38.46})

In [68]:
kwargs = {
    "dataset": "Gutural Scream and Metal Vocals",  # a 'pretty' name for the training dataset
    "dataset_args": "split: test",
    "language": "en",
    "model_name": "Whisper Tiny Metal - Juan Pablo Diaz",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard"
}

In [69]:
trainer.push_to_hub(**kwargs)

'https://huggingface.co/jpdiazpardo/whisper-tiny-metal/tree/main/'